### Build testgen Python Package

In [ ]:
# !pip -q install -e ../.

### Imports

In [1]:
import pandas as pd
import json
import os
from tqdm.notebook import tqdm
from testgen.utils import *
from pathlib import Path
from datetime import datetime as dt
from dotenv import load_dotenv

### EDA

In [2]:
base_path = Path().cwd().parent
load_dotenv()

True

In [44]:
# read data and rename columns
df = pd.read_excel(base_path / "data/requirements.xlsx")
df_examples = pd.read_excel(base_path / "data/examples.xlsx")

In [45]:
# Rename columns with uppercase first letters of each word
df.columns = [''.join(word[0].capitalize() for word in col.split('_')) for col in df.columns]

In [4]:
# make sure all columns except the first one are binary
for c in df.columns[1:]:
    assert df[c].drop_duplicates().shape[0] == 2

# Find Examples

In [ ]:
N_EXAMPLES = 1


if N_EXAMPLES > 1:
    indexes_to_drop, examples = get_examples_from_df(df_examples.drop(columns=["selected"]), N_EXAMPLES)


if N_EXAMPLES == 1: # pick pre selected example when N_EXAMPLES = 1
    df_examples_n1 = df_examples[df_examples["selected"] == 1].copy()
    df_examples_n1.drop(columns=["selected"], inplace=True)
    indexes_to_drop, examples = get_examples_from_df(df_examples_n1, 1)

In [60]:
# join all examples in a text format to add to prompt
examples_txt = ""

for e1 in examples.values():
    for e2 in e1:
        examples_txt += f"Requirement: {e2[0]}\n"
        examples_txt += f"Vector: {e2[1]}\n"
        examples_txt += f"Target Sensor/s: {' and '.join(df.columns[1:][[True if x == 1 else False for x in map(int, e2[1][1:-1].split(','))]])}\n"
        examples_txt += "\n"

In [61]:
print("\n".join(examples_txt.split("\n")[-9:]))

Requirement: The power steering system must adapt the torque levels in response to detected road surface conditions (e.g., ice, water, gravel)
Vector: [0,0,0,0,1]
Target Sensor/s: steering_torque

Requirement: The control system must detect and mitigate power oversteer to maintain vehicle control by adjusting the wheel steering angle and acceleration pedal inputs
Vector: [1,1,0,0,0]
Target Sensor/s: acceleration_pedal and wheel_steering_angle




# LLM

In [62]:
from testgen.prompts import SystemPrompt
from testgen.prompts import Sensors
from testgen.prompts import UserPrompt

In [ ]:
llm_models = {
    "azure": [os.getenv("AZURE_DEPLOYMENT_NAME")],
    "groq": [
        "llama-3.2-3b-preview",
        "llama-3.2-90b-text-preview",
        "mixtral-8x7b-32768",
        "llama-3.2-11b-text-preview"
    ],
}

endpoint_attrs = {
    "azure": {
        "api_key": os.getenv("AZURE_OPENAI_API_KEY"),
        "api_version": os.getenv("AZURE_API_VERSION"),
        "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    },
    "groq": {
        "api_key": os.getenv("GROQ_API_KEY"),
    },
}

In [64]:
endpoint_name = "groq"
model_name = llm_models[endpoint_name][3]

client = llm_client(endpoint_name, **endpoint_attrs[endpoint_name])

In [65]:
# t = client.chat.completions.create(
#     model=model_name,
#     messages=[{"role": "user", "content": "Respond: Here"}]
# )
# t.choices[0].message.content

# Run for all Requirements

In [66]:
results = []
responses = []

for instance in tqdm(df.iterrows()):
    result, response = client_invoke(
        client,
        model_name,
        SystemPrompt,
        Sensors,
        examples_txt,
        UserPrompt,
        instance,
    )

    results.append(result)
    responses.append(response)

0it [00:00, ?it/s]

In [67]:
accuracy = 0
total_tokens = 0
total_completion_tokens = 0
total_time = 0

for r in results:
    accuracy += r["accuracy"]
    total_tokens += r["total_tokens"]
    total_completion_tokens += r["completion_tokens"]
    total_time += r['response_time']

number_of_reqs = len(results)
accuracy /= len(results)
avg_time_per_req = round(total_time / len(results), 6)
avg_token_per_req = total_tokens / len(results)
avg_completion_token_per_req = total_completion_tokens / len(results)

In [68]:
# save results
time = dt.now()

results_path = "results/single_{model}_n-{examples}_acc-{accuracy}_{time}.json"
results_path = results_path.format(
    model=model_name,
    examples=N_EXAMPLES,
    time=time.strftime('%m.%d.%Y-%H:%M:%S'),
    accuracy=round(accuracy, 3)
)

results_file = base_path / results_path
results_file.parent.mkdir(exist_ok=True)
results_file.touch()

with results_file.open("w") as f:
    json.dump({"accuracy": accuracy,
        "number_of_reqs": number_of_reqs,
        "total_tokens": total_tokens,
        "total_completion_tokens": total_completion_tokens,
        "avg_token_per_req": avg_token_per_req,
        "avg_completion_token_per_req": avg_completion_token_per_req,
        "avg_time_per_req": avg_time_per_req,
        "examples": examples,
        "responses": results}, f, indent=4)